In [3]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
import joblib
from scipy.special import softmax
import sasc.config
import numpy as np
from collections import defaultdict
from copy import deepcopy
import pandas as pd
# import story_helper
from sasc.modules.fmri_module import convert_module_num_to_voxel_num
from sasc.config import FMRI_DIR

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# load stuff
# double check all of these, intro paragraph may be the same...
story_mapping = {
    "interactions/uts02___jun14___seed=1": "GenStory7_resps.npy",
    "interactions/uts02___jun14___seed=4": "GenStory8_resps.npy",
    "polysemantic/uts02___jun14___seed=6": "GenStory9_resps.npy",
    "polysemantic/uts02___jun14___seed=1": "GenStory10_resps.npy",
}

STORIES_DIR = join(sasc.config.RESULTS_DIR, "stories")
story_names = story_mapping.keys()  # os.listdir(STORIES_DIR)
# cluster_neighbors = joblib.load(join(FMRI_DIR, "voxel_neighbors_and_pcs", "cluster_neighbors_v1.pkl"))
perfs = joblib.load(join(sasc.config.FMRI_DIR, 'rj_models', 'opt_model', 'new_setup_performance.jbl'))

# process with timings
story_data = defaultdict(list)
for story_idx, story_name in enumerate(story_names):
    story_data["timing"].append(
        pd.read_csv(join(STORIES_DIR, story_name, "timings_processed.csv"))
    )
    story_data["story_name_original"].append(story_name)
    story_data["story_name_new"].append(story_mapping[story_name])
    story_data["story_text"].append(
        open(join(STORIES_DIR, story_name, "story.txt"), "r").read()
    )
    prompts_paragraphs = joblib.load(
        join(STORIES_DIR, story_name, "prompts_paragraphs.pkl")
    )
    story_data["prompts"].append(prompts_paragraphs["prompts"])
    story_data["paragraphs"].append(prompts_paragraphs["paragraphs"])

    # rows
    # rows = pd.read_csv(join(STORIES_DIR, story_name, "rows.csv"))
    rows = pd.read_pickle(join(STORIES_DIR, story_name, "rows.pkl"))
    rows["voxel_num"] = rows.apply(
        lambda row: convert_module_num_to_voxel_num(row["module_num"], row["subject"]),
        axis=1,
    )
    rows = rows[
        [
            "expl",
            "module_num",
            "top_explanation_init_strs",
            "subject",
            "fmri_test_corr",
            "top_score_synthetic",
            "roi_anat",
            "roi_func",
            "voxel_num",
        ]
    ]
    rows['test_corr_new'] = rows['voxel_num'].apply(lambda x: perfs[x])
    # rows['cluster_nums'] = rows['voxel_num'].map(cluster_neighbors)
    story_data["rows"].append(rows)

    if "interactions" in list(story_mapping.keys())[story_idx]:
        rows1 = pd.read_pickle(join(STORIES_DIR, story_name, "rows1.pkl"))
        rows2 = pd.read_pickle(join(STORIES_DIR, story_name, "rows2.pkl"))
        rows1["voxel_num"] = rows1.apply(
            lambda row: convert_module_num_to_voxel_num(
                row["module_num"], row["subject"]
            ),
            axis=1,
        )
        rows2["voxel_num"] = rows2.apply(
            lambda row: convert_module_num_to_voxel_num(
                row["module_num"], row["subject"]
            ),
            axis=1,
        )
        story_data['voxel_num1'].append(rows1['voxel_num'])
        story_data['voxel_num2'].append(rows2['voxel_num'])
        story_data['expl1'].append(rows1['expl'])
        story_data['expl2'].append(rows2['expl'])


joblib.dump(story_data, join(sasc.config.RESULTS_DIR, "pilot2_story_data.pkl"))

['/home/chansingh/automated-explanations/results/pilot2_story_data.pkl']